<a href="https://colab.research.google.com/github/bmwenyemali/smartcoinsDataAnalysis/blob/main/smartcoins_sql_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SmartCoins SQL Analysis Portfolio

**Author:** Bienvenu M Mwenyemali  
**Skills Demonstrated:** SQL, SQLite, Data Analysis, Business Intelligence

---

## Overview
This notebook demonstrates SQL analysis skills using cryptocurrency data from SmartCoins App.  
We use Python's sqlite3 to run SQL queries on our dataset.

**SQL Skills Covered:**
- SELECT, WHERE, ORDER BY
- Aggregate functions (COUNT, SUM, AVG, MIN, MAX)
- GROUP BY and HAVING
- CASE statements
- Subqueries
- Window functions (RANK, ROW_NUMBER)
- Common Table Expressions (CTEs)

## Section 1: Setup and Data Loading

In [1]:
import sqlite3
import pandas as pd
import requests
from datetime import datetime

print("Libraries imported successfully!")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Libraries imported successfully!
Analysis Date: 2026-02-11 16:32:20


In [2]:
# Fetch data from API
API_URL = "https://smartcoinsapp.com/api/coins"
print(f"Fetching data from: {API_URL}")

response = requests.get(API_URL, timeout=30)
data = response.json()
print(f"Retrieved {len(data)} coins")

Fetching data from: https://smartcoinsapp.com/api/coins
Retrieved 5 coins


In [5]:
records = []
for coin in data['data']:
    scores = coin.get('investmentScores', {})
    risk = coin.get('riskMetrics', {})

    # Handle 'signals' which can be a dict or a list
    signals_data = coin.get('signals', {})
    primary_signal_val = 'NEUTRAL'
    signal_strength_val = 0
    overall_score_val = 0
    composite_score_val = 0
    change_momentum_val = 0

    if isinstance(signals_data, dict):
        primary_signal_val = signals_data.get('primarySignal', 'NEUTRAL')
        signal_strength_val = signals_data.get('signalStrength', 0)
        overall_score_val = signals_data.get('overallScore', 0)
        composite_score_val = signals_data.get('compositeScore', 0)
        change_momentum_val = signals_data.get('changeMomentum', 0)
    elif isinstance(signals_data, list) and signals_data:
        primary_signal_val = signals_data[0] # Assuming first element is the primary signal
        # Other signal-related metrics like strength, score, momentum are likely not present when signals_data is a list
        # So they will remain at their default values (0).

    record = {
        'coin_name': coin.get('name'),
        'symbol': coin.get('symbol'),
        'price_usd': coin.get('price', 0), # Direct from coin, not quote
        'market_cap': coin.get('marketCap', 0), # Direct from coin, changed key name
        'volume_24h': coin.get('volume24h', 0), # Direct from coin, changed key name
        'pct_change_24h': coin.get('percentChange24h', 0), # Direct from coin, changed key name
        'pct_change_7d': coin.get('percentChange7d', 0), # Direct from coin, changed key name
        'pct_change_30d': coin.get('percentChange30d', 0), # Direct from coin, changed key name
        'coin_type': coin.get('coinType', 'Unknown'),
        'category': coin.get('category', 'Uncategorized'),
        'primary_signal': primary_signal_val,
        'signal_strength': signal_strength_val,
        'overall_score': overall_score_val,
        'composite_score': composite_score_val,
        'change_momentum': change_momentum_val,
        'price_volatility': risk.get('priceVolatility', 0),
        'volatility_risk': risk.get('volatilityRisk', 0),
        'liquidity_risk': risk.get('liquidityRisk', 0),
        'inv_momentum_score': scores.get('momentumScore', 0),
        'inv_risk_score': scores.get('riskScore', 0),
        'circulating_supply': coin.get('circulating_supply', 0),
    }
    records.append(record)

df = pd.DataFrame(records)
df = df.drop_duplicates(subset=['symbol'], keep='first')
print(f"DataFrame: {len(df)} rows, {len(df.columns)} columns")

DataFrame: 93 rows, 21 columns


In [6]:
# Create SQLite database in memory
conn = sqlite3.connect(':memory:')
df.to_sql('coins', conn, if_exists='replace', index=False)
print("Database created successfully!")

# Helper function to run SQL and display results
def run_sql(query, description=""):
    if description:
        print(f"\n{description}")
        print("-" * 50)
    result = pd.read_sql_query(query, conn)
    return result

Database created successfully!


## Section 2: Basic SELECT Queries

In [7]:
# Query 1: Select all columns, first 10 rows
query = """
SELECT *
FROM coins
LIMIT 10;
"""
run_sql(query, "First 10 coins - All columns")


First 10 coins - All columns
--------------------------------------------------


,coin_name,symbol,price_usd,market_cap,volume_24h,pct_change_24h,pct_change_7d,pct_change_30d,coin_type,category,...,signal_strength,overall_score,composite_score,change_momentum,price_volatility,volatility_risk,liquidity_risk,inv_momentum_score,inv_risk_score,circulating_supply
0,Moonchain,MCH,0.018842,4.700058e+06,1.087804e+05,-0.664301,203866.282762,1.533105e+04,meme,Small,...,0,0,0,0,0,0,0,0,0,0
1,Smilek to the Bank,SMILEK,0.000012,2.393957e+07,2.151252e+04,1384.381588,161561.339835,1.104888e+05,crypto,Small,...,0,0,0,0,0,0,0,0,0,0
2,Donkey King,DOKY,0.001699,1.473476e+07,0.000000e+00,2051.951564,184505.231044,1.282338e+05,meme,Small,...,0,0,0,0,0,0,0,0,0,0
3,Beers,BEER,285832.660071,0.000000e+00,3.156100e+05,1686.319242,56970.664581,1.524265e+09,meme,Small,...,0,0,0,0,0,0,0,0,0,0
4,APES,APES,0.006916,6.916416e+06,1.121421e+04,2943.119548,39134.287204,2.228922e+04,crypto,Small,...,0,0,0,0,0,0,0,0,0,0
5,MAGA,PEOPLE,3.066520,0.000000e+00,1.278658e+05,1888.587208,32156.777051,6.943154e+06,meme,Small,...,0,0,0,0,0,0,0,0,0,0
6,GameStop Coin,GME,432.054755,0.000000e+00,2.207821e+05,1249.682108,28589.004820,5.121417e+07,meme,Small,...,0,0,0,0,0,0,0,0,0,0
7,Wojak (wojaketh),WOJAK,0.000030,2.069284e+06,7.619460e+06,-29.424754,18009.740050,-7.777978e+01,meme,Small,...,0,0,0,0,0,0,0,0,0,0
8,KnockOut Games,GG,0.002805,2.805005e+06,0.000000e+00,-4.494759,21027.854396,1.606634e+01,crypto,Small,...,0,0,0,0,0,0,0,0,0,0
9,NOT,CAT,15780.574668,0.000000e+00,2.120899e+05,1416.065258,11030.540116,1.393765e+07,meme,Small,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Query 2: Select specific columns
query = """
SELECT coin_name, symbol, price_usd, market_cap, primary_signal
FROM coins
LIMIT 10;
"""
run_sql(query, "Select specific columns")


Select specific columns
--------------------------------------------------


,coin_name,symbol,price_usd,market_cap,primary_signal
0,Moonchain,MCH,0.018842,4.700058e+06,STRONG_BUY
1,Smilek to the Bank,SMILEK,0.000012,2.393957e+07,STRONG_BUY
2,Donkey King,DOKY,0.001699,1.473476e+07,BUY
3,Beers,BEER,285832.660071,0.000000e+00,STRONG_BUY
4,APES,APES,0.006916,6.916416e+06,STRONG_BUY
5,MAGA,PEOPLE,3.066520,0.000000e+00,STRONG_BUY
6,GameStop Coin,GME,432.054755,0.000000e+00,STRONG_BUY
7,Wojak (wojaketh),WOJAK,0.000030,2.069284e+06,STRONG_BUY
8,KnockOut Games,GG,0.002805,2.805005e+06,BUY
9,NOT,CAT,15780.574668,0.000000e+00,STRONG_BUY


In [9]:
# Query 3: Count total records
query = """
SELECT COUNT(*) AS total_coins
FROM coins;
"""
run_sql(query, "Total number of coins")


Total number of coins
--------------------------------------------------


,total_coins
0,93


## Section 3: Filtering with WHERE

In [10]:
# Query 4: Filter by signal
query = """
SELECT coin_name, symbol, price_usd, primary_signal, overall_score
FROM coins
WHERE primary_signal = 'BUY'
ORDER BY overall_score DESC;
"""
run_sql(query, "Coins with BUY signal")


Coins with BUY signal
--------------------------------------------------


,coin_name,symbol,price_usd,primary_signal,overall_score
0,Donkey King,DOKY,0.001699,BUY,0
1,KnockOut Games,GG,0.002805,BUY,0
2,NULS,NULS,0.003598,BUY,0
3,Groestlcoin,GRS,0.091642,BUY,0
4,Robinhood,HOOD,0.000359,BUY,0
5,KIRA,KEX,0.003275,BUY,0
6,PENGU AI,PENGU,0.000007,BUY,0
7,apM Coin,APM,0.000661,BUY,0
8,CyberFi Token,CFi,0.082019,BUY,0
9,Veloce,VEXT,0.000100,BUY,0


In [11]:
# Query 5: Multiple conditions with AND
query = """
SELECT coin_name, symbol, price_usd, volatility_risk, overall_score
FROM coins
WHERE overall_score > 60
  AND volatility_risk < 2
ORDER BY overall_score DESC;
"""
run_sql(query, "High score AND low risk coins")


High score AND low risk coins
--------------------------------------------------


,coin_name,symbol,price_usd,volatility_risk,overall_score


In [12]:
# Query 6: Filter with OR and IN
query = """
SELECT coin_name, symbol, primary_signal, overall_score
FROM coins
WHERE primary_signal IN ('BUY', 'STRONG BUY')
   OR overall_score > 70
ORDER BY overall_score DESC
LIMIT 15;
"""
run_sql(query, "BUY signals OR high overall score")


BUY signals OR high overall score
--------------------------------------------------


,coin_name,symbol,primary_signal,overall_score
0,Donkey King,DOKY,BUY,0
1,KnockOut Games,GG,BUY,0
2,NULS,NULS,BUY,0
3,Groestlcoin,GRS,BUY,0
4,Robinhood,HOOD,BUY,0
5,KIRA,KEX,BUY,0
6,PENGU AI,PENGU,BUY,0
7,apM Coin,APM,BUY,0
8,CyberFi Token,CFi,BUY,0
9,Veloce,VEXT,BUY,0


In [13]:
# Query 7: Filter with BETWEEN
query = """
SELECT coin_name, symbol, price_usd, overall_score
FROM coins
WHERE price_usd BETWEEN 0.001 AND 1
ORDER BY price_usd DESC;
"""
run_sql(query, "Coins priced between $0.001 and $1")


Coins priced between $0.001 and $1
--------------------------------------------------


,coin_name,symbol,price_usd,overall_score
0,MSTR,MSTR,0.874090,0
1,Robonomics.network,XRT,0.499412,0
2,pippin,PIPPIN,0.402552,0
3,Power Protocol,POWER,0.373463,0
4,Huobi Token,HT,0.229680,0
5,Groestlcoin,GRS,0.091642,0
6,Billards Hub,8BALL,0.089656,0
7,CyberFi Token,CFi,0.082019,0
8,Collect on Fanable,COLLECT,0.071300,0
9,Reploy,RAI,0.068195,0


## Section 4: Sorting with ORDER BY

In [14]:
# Query 8: Top 10 by market cap
query = """
SELECT coin_name, symbol, market_cap, volume_24h
FROM coins
WHERE market_cap > 0
ORDER BY market_cap DESC
LIMIT 10;
"""
run_sql(query, "Top 10 coins by Market Cap")


Top 10 coins by Market Cap
--------------------------------------------------


,coin_name,symbol,market_cap,volume_24h
0,pippin,PIPPIN,4.025501e+08,6.245588e+07
1,Power Protocol,POWER,7.842730e+07,6.257912e+07
2,WAR,WAR,4.053365e+07,1.148109e+07
3,Collect on Fanable,COLLECT,3.828805e+07,1.289055e+07
4,Rifampicin,RIF,2.619660e+07,8.916620e+05
5,Smilek to the Bank,SMILEK,2.393957e+07,2.151252e+04
6,Donkey King,DOKY,1.473476e+07,0.000000e+00
7,saffron.finance,SFI,1.259599e+07,7.743729e+04
8,ShibaBitcoin,SBBTC,1.165126e+07,7.079578e+04
9,NKN,NKN,1.021590e+07,2.528981e+07


In [15]:
# Query 9: Top 10 by overall score
query = """
SELECT coin_name, symbol, overall_score, composite_score, primary_signal
FROM coins
ORDER BY overall_score DESC
LIMIT 10;
"""
run_sql(query, "Top 10 coins by Overall Score")


Top 10 coins by Overall Score
--------------------------------------------------


,coin_name,symbol,overall_score,composite_score,primary_signal
0,Moonchain,MCH,0,0,STRONG_BUY
1,Smilek to the Bank,SMILEK,0,0,STRONG_BUY
2,Donkey King,DOKY,0,0,BUY
3,Beers,BEER,0,0,STRONG_BUY
4,APES,APES,0,0,STRONG_BUY
5,MAGA,PEOPLE,0,0,STRONG_BUY
6,GameStop Coin,GME,0,0,STRONG_BUY
7,Wojak (wojaketh),WOJAK,0,0,STRONG_BUY
8,KnockOut Games,GG,0,0,BUY
9,NOT,CAT,0,0,STRONG_BUY


In [16]:
# Query 10: Bottom 10 by volatility risk (lowest risk)
query = """
SELECT coin_name, symbol, volatility_risk, liquidity_risk, primary_signal
FROM coins
ORDER BY volatility_risk ASC
LIMIT 10;
"""
run_sql(query, "Top 10 Lowest Risk coins")


Top 10 Lowest Risk coins
--------------------------------------------------


,coin_name,symbol,volatility_risk,liquidity_risk,primary_signal
0,Moonchain,MCH,0,0,STRONG_BUY
1,Smilek to the Bank,SMILEK,0,0,STRONG_BUY
2,Donkey King,DOKY,0,0,BUY
3,Beers,BEER,0,0,STRONG_BUY
4,APES,APES,0,0,STRONG_BUY
5,MAGA,PEOPLE,0,0,STRONG_BUY
6,GameStop Coin,GME,0,0,STRONG_BUY
7,Wojak (wojaketh),WOJAK,0,0,STRONG_BUY
8,KnockOut Games,GG,0,0,BUY
9,NOT,CAT,0,0,STRONG_BUY


## Section 5: Aggregate Functions

In [17]:
# Query 11: Basic aggregates
query = """
SELECT
    COUNT(*) AS total_coins,
    ROUND(AVG(overall_score), 2) AS avg_score,
    ROUND(MIN(overall_score), 2) AS min_score,
    ROUND(MAX(overall_score), 2) AS max_score,
    ROUND(AVG(volatility_risk), 2) AS avg_risk
FROM coins;
"""
run_sql(query, "Summary Statistics")


Summary Statistics
--------------------------------------------------


,total_coins,avg_score,min_score,max_score,avg_risk
0,93,0.0,0.0,0.0,0.0


In [18]:
# Query 12: Sum and average of market metrics
query = """
SELECT
    ROUND(SUM(market_cap), 2) AS total_market_cap,
    ROUND(AVG(market_cap), 2) AS avg_market_cap,
    ROUND(SUM(volume_24h), 2) AS total_volume,
    ROUND(AVG(volume_24h), 2) AS avg_volume
FROM coins
WHERE market_cap > 0;
"""
run_sql(query, "Market Metrics Summary")


Market Metrics Summary
--------------------------------------------------


,total_market_cap,avg_market_cap,total_volume,avg_volume
0,7.281469e+08,16933648.43,1.889727e+08,4394713.04


## Section 6: GROUP BY Analysis

In [19]:
# Query 13: Count by signal type
query = """
SELECT
    primary_signal,
    COUNT(*) AS coin_count,
    ROUND(AVG(overall_score), 2) AS avg_score
FROM coins
GROUP BY primary_signal
ORDER BY coin_count DESC;
"""
run_sql(query, "Coins grouped by Signal")


Coins grouped by Signal
--------------------------------------------------


,primary_signal,coin_count,avg_score
0,STRONG_BUY,82,0.0
1,BUY,11,0.0


In [20]:
# Query 14: Count by coin type
query = """
SELECT
    coin_type,
    COUNT(*) AS coin_count,
    ROUND(AVG(overall_score), 2) AS avg_score,
    ROUND(AVG(volatility_risk), 2) AS avg_risk
FROM coins
GROUP BY coin_type
ORDER BY coin_count DESC;
"""
run_sql(query, "Analysis by Coin Type")


Analysis by Coin Type
--------------------------------------------------


,coin_type,coin_count,avg_score,avg_risk
0,crypto,59,0.0,0.0
1,meme,34,0.0,0.0


In [21]:
# Query 15: GROUP BY with HAVING
query = """
SELECT
    coin_type,
    COUNT(*) AS coin_count,
    ROUND(AVG(overall_score), 2) AS avg_score
FROM coins
GROUP BY coin_type
HAVING COUNT(*) >= 3
ORDER BY avg_score DESC;
"""
run_sql(query, "Coin types with 3+ coins (HAVING)")


Coin types with 3+ coins (HAVING)
--------------------------------------------------


,coin_type,coin_count,avg_score
0,meme,34,0.0
1,crypto,59,0.0


## Section 7: CASE Statements

In [22]:
# Query 16: Create price tier using CASE
query = """
SELECT
    coin_name,
    symbol,
    price_usd,
    CASE
        WHEN price_usd < 0.001 THEN 'Micro'
        WHEN price_usd < 1 THEN 'Low'
        WHEN price_usd < 100 THEN 'Medium'
        ELSE 'High'
    END AS price_tier
FROM coins
ORDER BY price_usd DESC
LIMIT 15;
"""
run_sql(query, "Price Tier Classification (CASE)")


Price Tier Classification (CASE)
--------------------------------------------------


,coin_name,symbol,price_usd,price_tier
0,Beers,BEER,285832.660071,High
1,XenBitcoin,XBTC,69329.832202,High
2,NOT,CAT,15780.574668,High
3,GameStop Coin,GME,432.054755,High
4,saffron.finance,SFI,137.776262,High
5,ShibaBitcoin,SBBTC,99.939121,Medium
6,Tesla (buytesla.net),TSLA,77.476142,Medium
7,[Fake]COINBASE,COIN,35.634765,Medium
8,MAGA,PEOPLE,3.066520,Medium
9,Splendor,SPLD,2.483026,Medium


In [23]:
# Query 17: Risk level classification
query = """
SELECT
    coin_name,
    symbol,
    volatility_risk,
    CASE
        WHEN volatility_risk < 0.5 THEN 'Low Risk'
        WHEN volatility_risk < 2 THEN 'Medium Risk'
        ELSE 'High Risk'
    END AS risk_level
FROM coins
ORDER BY volatility_risk ASC
LIMIT 15;
"""
run_sql(query, "Risk Level Classification")


Risk Level Classification
--------------------------------------------------


,coin_name,symbol,volatility_risk,risk_level
0,Moonchain,MCH,0,Low Risk
1,Smilek to the Bank,SMILEK,0,Low Risk
2,Donkey King,DOKY,0,Low Risk
3,Beers,BEER,0,Low Risk
4,APES,APES,0,Low Risk
5,MAGA,PEOPLE,0,Low Risk
6,GameStop Coin,GME,0,Low Risk
7,Wojak (wojaketh),WOJAK,0,Low Risk
8,KnockOut Games,GG,0,Low Risk
9,NOT,CAT,0,Low Risk


In [24]:
# Query 18: Count by risk level
query = """
SELECT
    CASE
        WHEN volatility_risk < 0.5 THEN 'Low Risk'
        WHEN volatility_risk < 2 THEN 'Medium Risk'
        ELSE 'High Risk'
    END AS risk_level,
    COUNT(*) AS coin_count,
    ROUND(AVG(overall_score), 2) AS avg_score
FROM coins
GROUP BY risk_level
ORDER BY coin_count DESC;
"""
run_sql(query, "Distribution by Risk Level")


Distribution by Risk Level
--------------------------------------------------


,risk_level,coin_count,avg_score
0,Low Risk,93,0.0


## Section 8: Subqueries

In [25]:
# Query 19: Coins above average score
query = """
SELECT coin_name, symbol, overall_score
FROM coins
WHERE overall_score > (SELECT AVG(overall_score) FROM coins)
ORDER BY overall_score DESC;
"""
run_sql(query, "Coins above average score (Subquery)")


Coins above average score (Subquery)
--------------------------------------------------


,coin_name,symbol,overall_score


In [26]:
# Query 20: Coins with lowest risk in each type
query = """
SELECT coin_name, symbol, coin_type, volatility_risk
FROM coins c1
WHERE volatility_risk = (
    SELECT MIN(volatility_risk)
    FROM coins c2
    WHERE c2.coin_type = c1.coin_type
)
ORDER BY coin_type;
"""
run_sql(query, "Lowest risk coin per type (Correlated Subquery)")


Lowest risk coin per type (Correlated Subquery)
--------------------------------------------------


,coin_name,symbol,coin_type,volatility_risk
0,Smilek to the Bank,SMILEK,crypto,0
1,APES,APES,crypto,0
2,KnockOut Games,GG,crypto,0
3,Matr1x,MAX,crypto,0
4,The Game Company,GMRT,crypto,0
...,...,...,...,...
88,Formula 1,CHAMPAGNE,meme,0
89,ShibaBitcoin,SBBTC,meme,0
90,TRI SIGMA,TRISIG,meme,0
91,Hachiko Inu,HACHIKO,meme,0


## Section 9: Window Functions

In [27]:
# Query 21: RANK by overall score
query = """
SELECT
    RANK() OVER (ORDER BY overall_score DESC) AS rank,
    coin_name,
    symbol,
    overall_score,
    primary_signal
FROM coins
LIMIT 15;
"""
run_sql(query, "Ranked by Overall Score (RANK)")


Ranked by Overall Score (RANK)
--------------------------------------------------


,rank,coin_name,symbol,overall_score,primary_signal
0,1,Moonchain,MCH,0,STRONG_BUY
1,1,Smilek to the Bank,SMILEK,0,STRONG_BUY
2,1,Donkey King,DOKY,0,BUY
3,1,Beers,BEER,0,STRONG_BUY
4,1,APES,APES,0,STRONG_BUY
5,1,MAGA,PEOPLE,0,STRONG_BUY
6,1,GameStop Coin,GME,0,STRONG_BUY
7,1,Wojak (wojaketh),WOJAK,0,STRONG_BUY
8,1,KnockOut Games,GG,0,BUY
9,1,NOT,CAT,0,STRONG_BUY


In [28]:
# Query 22: ROW_NUMBER within coin type
query = """
SELECT
    coin_type,
    ROW_NUMBER() OVER (PARTITION BY coin_type ORDER BY overall_score DESC) AS type_rank,
    coin_name,
    symbol,
    overall_score
FROM coins
WHERE coin_type IN ('token', 'coin')
LIMIT 20;
"""
run_sql(query, "Ranking within coin type (PARTITION BY)")


Ranking within coin type (PARTITION BY)
--------------------------------------------------


,coin_type,type_rank,coin_name,symbol,overall_score


In [29]:
# Query 23: Running total of market cap
query = """
SELECT
    coin_name,
    symbol,
    market_cap,
    SUM(market_cap) OVER (ORDER BY market_cap DESC) AS running_total
FROM coins
WHERE market_cap > 0
LIMIT 10;
"""
run_sql(query, "Running Total of Market Cap")


Running Total of Market Cap
--------------------------------------------------


,coin_name,symbol,market_cap,running_total
0,pippin,PIPPIN,4.025501e+08,4.025501e+08
1,Power Protocol,POWER,7.842730e+07,4.809774e+08
2,WAR,WAR,4.053365e+07,5.215110e+08
3,Collect on Fanable,COLLECT,3.828805e+07,5.597991e+08
4,Rifampicin,RIF,2.619660e+07,5.859957e+08
5,Smilek to the Bank,SMILEK,2.393957e+07,6.099353e+08
6,Donkey King,DOKY,1.473476e+07,6.246700e+08
7,saffron.finance,SFI,1.259599e+07,6.372660e+08
8,ShibaBitcoin,SBBTC,1.165126e+07,6.489173e+08
9,NKN,NKN,1.021590e+07,6.591332e+08


## Section 10: Common Table Expressions (CTEs)

In [30]:
# Query 24: CTE for top performers analysis
query = """
WITH top_performers AS (
    SELECT coin_name, symbol, overall_score, volatility_risk
    FROM coins
    WHERE overall_score > 60
),
low_risk AS (
    SELECT coin_name, symbol, overall_score, volatility_risk
    FROM top_performers
    WHERE volatility_risk < 2
)
SELECT * FROM low_risk
ORDER BY overall_score DESC;
"""
run_sql(query, "High performers with low risk (CTE)")


High performers with low risk (CTE)
--------------------------------------------------


,coin_name,symbol,overall_score,volatility_risk


In [31]:
# Query 25: CTE with aggregation
query = """
WITH signal_stats AS (
    SELECT
        primary_signal,
        COUNT(*) AS count,
        ROUND(AVG(overall_score), 2) AS avg_score,
        ROUND(AVG(volatility_risk), 2) AS avg_risk
    FROM coins
    GROUP BY primary_signal
)
SELECT
    primary_signal,
    count,
    avg_score,
    avg_risk,
    ROUND(100.0 * count / SUM(count) OVER (), 2) AS percentage
FROM signal_stats
ORDER BY count DESC;
"""
run_sql(query, "Signal distribution with percentage (CTE)")


Signal distribution with percentage (CTE)
--------------------------------------------------


,primary_signal,count,avg_score,avg_risk,percentage
0,STRONG_BUY,82,0.0,0.0,88.17
1,BUY,11,0.0,0.0,11.83


## Section 11: Business Intelligence Queries

In [32]:
# Query 26: Investment recommendation report
query = """
SELECT
    RANK() OVER (ORDER BY overall_score DESC) AS rank,
    coin_name,
    symbol,
    coin_type,
    ROUND(price_usd, 8) AS price,
    overall_score,
    primary_signal,
    CASE
        WHEN volatility_risk < 0.5 THEN 'Low'
        WHEN volatility_risk < 2 THEN 'Medium'
        ELSE 'High'
    END AS risk_level
FROM coins
WHERE overall_score > 50
ORDER BY overall_score DESC
LIMIT 20;
"""
run_sql(query, "Investment Recommendations Report")


Investment Recommendations Report
--------------------------------------------------


,rank,coin_name,symbol,coin_type,price,overall_score,primary_signal,risk_level


In [33]:
# Query 27: Market overview dashboard
query = """
SELECT
    'Total Coins' AS metric, CAST(COUNT(*) AS TEXT) AS value FROM coins
UNION ALL
SELECT
    'Avg Overall Score', CAST(ROUND(AVG(overall_score), 2) AS TEXT) FROM coins
UNION ALL
SELECT
    'BUY Signals', CAST(COUNT(*) AS TEXT) FROM coins WHERE primary_signal LIKE '%BUY%'
UNION ALL
SELECT
    'SELL Signals', CAST(COUNT(*) AS TEXT) FROM coins WHERE primary_signal LIKE '%SELL%'
UNION ALL
SELECT
    'Low Risk Coins', CAST(COUNT(*) AS TEXT) FROM coins WHERE volatility_risk < 0.5;
"""
run_sql(query, "Market Overview Dashboard")


Market Overview Dashboard
--------------------------------------------------


,metric,value
0,Total Coins,93
1,Avg Overall Score,0.0
2,BUY Signals,93
3,SELL Signals,0
4,Low Risk Coins,93


In [34]:
# Query 28: Cross-tabulation - Signal vs Risk
query = """
SELECT
    primary_signal,
    SUM(CASE WHEN volatility_risk < 0.5 THEN 1 ELSE 0 END) AS low_risk,
    SUM(CASE WHEN volatility_risk >= 0.5 AND volatility_risk < 2 THEN 1 ELSE 0 END) AS medium_risk,
    SUM(CASE WHEN volatility_risk >= 2 THEN 1 ELSE 0 END) AS high_risk,
    COUNT(*) AS total
FROM coins
GROUP BY primary_signal
ORDER BY total DESC;
"""
run_sql(query, "Signal vs Risk Level Cross-Tabulation")


Signal vs Risk Level Cross-Tabulation
--------------------------------------------------


,primary_signal,low_risk,medium_risk,high_risk,total
0,STRONG_BUY,82,0,0,82
1,BUY,11,0,0,11


## Section 12: Summary

In [35]:
print("=" * 60)
print("SQL ANALYSIS COMPLETE")
print("=" * 60)

print("\nSQL Skills Demonstrated:")
skills = [
    "SELECT, WHERE, ORDER BY, LIMIT",
    "Aggregate functions (COUNT, SUM, AVG, MIN, MAX)",
    "GROUP BY and HAVING clauses",
    "CASE statements for classification",
    "Subqueries (simple and correlated)",
    "Window functions (RANK, ROW_NUMBER, SUM OVER)",
    "Common Table Expressions (CTEs)",
    "UNION for combining results",
    "Cross-tabulation queries"
]
for skill in skills:
    print(f"  - {skill}")

print("\n" + "=" * 60)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Author: Bienvenu Mwenyemali")
print("=" * 60)

SQL ANALYSIS COMPLETE

SQL Skills Demonstrated:
  - SELECT, WHERE, ORDER BY, LIMIT
  - Aggregate functions (COUNT, SUM, AVG, MIN, MAX)
  - GROUP BY and HAVING clauses
  - CASE statements for classification
  - Subqueries (simple and correlated)
  - Window functions (RANK, ROW_NUMBER, SUM OVER)
  - Common Table Expressions (CTEs)
  - UNION for combining results
  - Cross-tabulation queries

Analysis Date: 2026-02-11 16:39:43
Author: Bienvenu Mwenyemali


In [36]:
# Close connection
conn.close()
print("Database connection closed.")

Database connection closed.
